In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from datetime import datetime

In [3]:
# Configuration
list_files = ['Accelerometer.csv', 'Gyroscope.csv', 'Location.csv']
data_dir = 'Data/train'
resample_interval = "1s"  # 100 milliseconds; use "1S" for 1 second
anchor_time = pd.Timestamp("2025-01-01 00:00")

# Store individual resampled dataframes
dataframes = {}

# Read and resample each file
for file_name in list_files:
    file_path = os.path.join(data_dir, file_name)
    if os.path.exists(file_path):
        key = file_name.replace('.csv', '')
        df = pd.read_csv(file_path)
        
        if 'Time (s)' not in df.columns:
            print(f"Skipping {file_name}: No 'Time (s)' column.")
            continue
        
        # Convert 'Time (s)' to timestamp
        df["timestamp"] = anchor_time + pd.to_timedelta(df["Time (s)"], unit="s")
        df.set_index("timestamp", inplace=True)
        df.sort_index(inplace=True)
        
        # Resample numeric columns only
        num_cols = df.select_dtypes(include="number").columns
        df_resampled = df[num_cols].resample(resample_interval).mean()

        dataframes[key] = df_resampled
        print(f"Loaded and resampled {file_name}")
    else:
        print(f"File not found: {file_name}")

# Merge all resampled dataframes on timestamp
combined_df = pd.concat(dataframes.values(), axis=1)

# Drop all leftover 'Time (s)' columns, if any
combined_df = combined_df.drop(columns=[col for col in combined_df.columns if col == 'Time (s)'])

# Reset index to range 0..N-1
combined_df.reset_index(drop=True, inplace=True)

# Show result
print("Combined DataFrame shape:", combined_df.shape)
print("Columns:", combined_df.columns.tolist())


Loaded and resampled Accelerometer.csv
Loaded and resampled Gyroscope.csv
Loaded and resampled Location.csv
Combined DataFrame shape: (579, 13)
Columns: ['X (m/s^2)', 'Y (m/s^2)', 'Z (m/s^2)', 'X (rad/s)', 'Y (rad/s)', 'Z (rad/s)', 'Latitude (°)', 'Longitude (°)', 'Height (m)', 'Velocity (m/s)', 'Direction (°)', 'Horizontal Accuracy (m)', 'Vertical Accuracy (°)']


In [4]:
combined_df.head()

,X (m/s^2),Y (m/s^2),Z (m/s^2),X (rad/s),Y (rad/s),Z (rad/s),Latitude (°),Longitude (°),Height (m),Velocity (m/s),Direction (°),Horizontal Accuracy (m),Vertical Accuracy (°)
0,-1.624422,4.472400,11.536510,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-1.177645,3.707805,8.383625,-0.165551,-0.039665,0.414780,52.165955,4.483189,7.596832,0.42,21.445312,6.014575,4.001463
2,-1.278969,2.368441,9.494935,-0.073334,0.184356,-0.110532,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-2.083857,3.067294,9.422469,0.015208,-0.044621,-0.007547,52.165973,4.483189,18.136501,0.20,NaN,11.162163,12.000000
4,-0.814126,2.924846,9.168069,-0.042164,0.003136,-0.066201,52.165976,4.483222,21.267361,0.20,40.078125,10.135889,12.000000
